## Imports

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [ ]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "umutakta"
    os.environ['KAGGLE_KEY']      = "a471e5474add8d57cadac3cc5ff2b1d6"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "/work/Strive_School/Feature_Engineering_(Module_4)/Day_2_ Data Cleaning/titanic/"

else:
    DATA_PATH = "/work/Data_Sets/titanic/"

DATA_PATH

'/work/Data_Sets/titanic/'

## Load data

In [ ]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [ ]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [ ]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

# YOUR CODE HERE


df['Title']      = df['Name'].map(get_Title_from_Name )
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [ ]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [ ]:
df["Title"] = df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

Extracting some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [ ]:
df.Cabin.fillna(0, inplace=True) 
df_test.Cabin.fillna(0, inplace=True) 

df.loc[df["Cabin"] != 0, "Cabin"] = 1
df_test.loc[df_test["Cabin"] != 0, "Cabin" ] =1

Filling **Age** null data taking mean value of Pclass-Sex 

In [ ]:
df.Age.fillna(df.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)
df_test.Age.fillna(df_test.groupby(['Pclass', 'Sex']).Age.transform('mean'), inplace=True)

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [ ]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [ ]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


Creating a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories)
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [ ]:
cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OneHotEncoder(
))
])


num_4_treemodels=pipeline.Pipeline(steps=[
    ("imputer",KNNImputer(n_neighbors=5)),
    ("quantile transformer",preprocessing.QuantileTransformer(n_quantiles=100, random_state=0))
    ])

tree_prepro = compose.ColumnTransformer(transformers=[
    ("cat mult",cat_4_multModels,cat_vars),
    ("num mult",num_4_multmodels,num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro



NameError: name 'cat_4_multModels' is not defined

In [ ]:
cat_4_multModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OneHotEncoder(
))
])


num_4_multmodels=pipeline.Pipeline(steps=[
    ("imputer",KNNImputer(n_neighbors=5)),
    ("quantile transformer",preprocessing.QuantileTransformer(n_quantiles=100, random_state=0))
    ])

tree_prepro = compose.ColumnTransformer(transformers=[
    ("cat mult",cat_4_multModels,cat_vars),
    ("num mult",num_4_multmodels,num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro



ColumnTransformer(transformers=[('cat mult',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('one hot', OneHotEncoder())]),
                                 ['Sex', 'Embarked', 'Title']),
                                ('num mult',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('quantile transformer',
                                                  QuantileTransformer(n_quantiles=100,
                                                                      random_state=0))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'])])

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [ ]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [ ]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
mult_classifiers={
    "SVM": svm.SVC(),
    "Logistic": LogisticRegression(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "MLP":MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)}

mult_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in mult_classifiers.items()}

mult_classifiers["SVM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age'])])),
                ('svc', SVC())])

In [ ]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in mult_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,Model,Accuracy,Bal Acc.,Time
1,MLP,84.357542,82.411067,0.213303
2,SVM,83.240223,80.421607,0.040903
3,Logistic,83.240223,81.501976,0.043592
4,KNN,81.005587,78.873518,0.078799


In [ ]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in mult_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (

,Model,Accuracy,Bal Acc.,Time
1,SVM,81.705948,78.595053,0.442440
2,MLP,81.705948,78.980922,1.747779
3,Logistic,81.257015,79.333237,0.765218
4,KNN,80.134680,78.532739,0.523471


In [ ]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": GradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier() }


# YOUR CODE HERE


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked', 'Title']),
                                                 ('num mult',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('quantile '
                                                                   'transformer',
                                                                   QuantileTransformer(n_quantiles=100,
                                                                                       random_state=0))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [ ]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

1:	learn: 0.6820801	total: 49.2ms	remaining: 24.5s
2:	learn: 0.6770935	total: 50.3ms	remaining: 16.7s
3:	learn: 0.6723015	total: 51.3ms	remaining: 12.8s
4:	learn: 0.6672715	total: 52.3ms	remaining: 10.4s
5:	learn: 0.6625852	total: 53.5ms	remaining: 8.86s
6:	learn: 0.6576968	total: 54.5ms	remaining: 7.73s
7:	learn: 0.6527901	total: 55.5ms	remaining: 6.89s
8:	learn: 0.6472399	total: 56.7ms	remaining: 6.24s
9:	learn: 0.6421550	total: 58ms	remaining: 5.74s
10:	learn: 0.6369511	total: 59.3ms	remaining: 5.33s
11:	learn: 0.6320371	total: 60.6ms	remaining: 4.99s
12:	learn: 0.6282963	total: 61.9ms	remaining: 4.7s
13:	learn: 0.6246123	total: 62.9ms	remaining: 4.43s
14:	learn: 0.6201889	total: 64.1ms	remaining: 4.21s
15:	learn: 0.6164933	total: 64.9ms	remaining: 3.99s
16:	learn: 0.6118215	total: 66.1ms	remaining: 3.82s
17:	learn: 0.6078144	total: 67.3ms	remaining: 3.67s
18:	learn: 0.6040424	total: 68.5ms	remaining: 3.54s
19:	learn: 0.5998107	total: 69.7ms	remaining: 3.42s
20:	learn: 0.5954308	tot

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,87.150838,85.764163,1.457878
2,Skl GBM,85.474860,83.860343,0.122993
3,Skl HistGBM,85.474860,83.860343,0.131135
4,AdaBoost,83.240223,82.852437,0.138458
5,LightGBM,83.240223,82.312253,0.098182
6,XGBoost,82.681564,81.857708,0.137700
7,Random Forest,82.122905,79.782609,0.195980
8,Extra Trees,80.446927,78.148880,0.177341
9,Decision Tree,78.212291,76.870883,0.025883


# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [ ]:

from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})





for model_name, model in tree_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              


# YOUR CODE HERE



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


1:	learn: 0.6801017	total: 2.24ms	remaining: 1.12s
2:	learn: 0.6748498	total: 3.35ms	remaining: 1.11s
3:	learn: 0.6714246	total: 3.94ms	remaining: 981ms
4:	learn: 0.6657903	total: 4.7ms	remaining: 936ms
5:	learn: 0.6600028	total: 5.58ms	remaining: 924ms
6:	learn: 0.6549985	total: 6.45ms	remaining: 915ms
7:	learn: 0.6502942	total: 7.63ms	remaining: 946ms
8:	learn: 0.6449366	total: 8.71ms	remaining: 959ms
9:	learn: 0.6408220	total: 9.42ms	remaining: 933ms
10:	learn: 0.6348829	total: 10.5ms	remaining: 943ms
11:	learn: 0.6304520	total: 11.2ms	remaining: 918ms
12:	learn: 0.6254857	total: 12ms	remaining: 912ms
13:	learn: 0.6208682	total: 13.1ms	remaining: 923ms
14:	learn: 0.6165273	total: 14.2ms	remaining: 931ms
15:	learn: 0.6110367	total: 15.3ms	remaining: 939ms
16:	learn: 0.6063400	total: 16.3ms	remaining: 944ms
17:	learn: 0.6019598	total: 17.4ms	remaining: 951ms
18:	learn: 0.5971015	total: 18.3ms	remaining: 945ms
19:	learn: 0.5935575	total: 19.2ms	remaining: 942ms
20:	learn: 0.5894821	tot

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.501684,81.485476,1.336409
2,Skl HistGBM,83.501684,81.485476,1.375561
3,CatBoost,82.940516,80.699358,13.613799
4,LightGBM,82.379349,80.629853,1.063948
5,XGBoost,81.930415,80.541175,1.600858
6,AdaBoost,81.705948,80.469274,1.134164
7,Random Forest,80.695847,78.932988,2.074267
8,Decision Tree,80.022447,78.331416,0.259393
9,Extra Trees,79.910213,78.405714,1.802408


**Training with all data the best model**

In [ ]:
best_model = tree_classifiers["XGBoost"]

# Fit best model with all data
best_model.fit(x,y)



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('xgbclassifier', XGBClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [ ]:
x_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        1
Cabin       0
Embarked    0
Title       1
dtype: int64

In [ ]:
test_pred = best_model.predict(x_test)


In [ ]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [ ]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0


In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100% 2.77k/2.77k [00:08<00:00, 338B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=796ba814-f632-4502-b0d7-7e2f8f9e546d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>